In [1]:
import sys
import os
import math
import moveit_commander
import moveit_msgs.msg
import rospy
import numpy as np
import trajectory_msgs.msg
from geometry_msgs.msg import PoseStamped
import tf
import actionlib
import subprocess

### ALL OF THESE IMPORTS MUST BE HANDLeD BY TAKESHI INSTALL

In [2]:
rospy.init_node('moveiteando')

In [3]:
listener = tf.TransformListener()

In [4]:
arm = moveit_commander.MoveGroupCommander('arm')
base=  moveit_commander.MoveGroupCommander('base')
gripper = moveit_commander.MoveGroupCommander('gripper')
head = moveit_commander.MoveGroupCommander('head')

In [7]:
def delete_object(name):
    u"""Gazeboの物体を消す関数

    引数:
        name (str): 物体の名前

    """

    cmd = ['rosservice', 'call', 'gazebo/delete_model',
           '{model_name: ' + str(name) + '}']
    subprocess.call(cmd)

    



In [8]:
#####SPAWN OBJECTS
from gazebo_ros import gazebo_interface
from geometry_msgs.msg import Pose, Quaternion
def spawn_object(gazebo_name, name, x, y, z, yaw):
    rospy.loginfo('Spawn: {0}'.format(name))
    initial_pose = Pose()
    initial_pose.position.x = x
    initial_pose.position.y = y
    initial_pose.position.z = z
    roll = 0.0
    pitch = 0.0
    yaw = 0.0
    q = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
    initial_pose.orientation = Quaternion(q[0], q[1], q[2], q[3])

    path_xml = _path_xml.replace('MODEL_NAME', name)

    with open(path_xml, "r") as f:
        model_xml = f.read()

    model_xml = model_xml.replace('PATH_TO_MODEL', _path_model)

    gazebo_interface.spawn_sdf_model_client(gazebo_name, model_xml, rospy.get_namespace(),
                                            initial_pose, "", "/gazebo")


In [9]:
## THIS PATH MUST BE SET BY USER ()

_path_xml = "/home/oscar/Codes/ycb_ws/src/robot_object_views/robotobject/models/MODEL_NAME/model-1_4.sdf"
_path_model = "/home/oscar/Codes/ycb_ws/src/robot_object_views/robotobject/models"

model_database_template = """<sdf version="1.4">
  <world name="default">
    <include>
      <uri>model://MODEL_NAME</uri>
    </include>
  </world>
</sdf>"""

objs=os.listdir(_path_model)
print (objs)

['073-b_lego_duplo', '065-i_cups', '065-f_cups', '065-g_cups', '063-b_marbles', '062_dice', '033_spatula', '050_medium_clamp', '007_tuna_fish_can', '059_chain', '052_extra_large_clamp', '065-e_cups', '043_phillips_screwdriver', '030_fork', '073-g_lego_duplo', '044_flat_screwdriver', '008_pudding_box', '051_large_clamp', '036_wood_block', '072-c_toy_airplane', '072-b_toy_airplane', '040_large_marker', '048_hammer', '058_golf_ball', '070-a_colored_wood_blocks', '065-h_cups', '061_foam_brick', '021_bleach_cleanser', '016_pear', '073-a_lego_duplo', '027_skillet', '055_baseball', '065-j_cups', '031_spoon', '028_skillet_lid', '014_lemon', '022_windex_bottle', '035_power_drill', '073-f_lego_duplo', '065-a_cups', '015_peach', '004_sugar_box', '032_knife', '006_mustard_bottle', '053_mini_soccer_ball', '071_nine_hole_peg_test', '026_sponge', '073-e_lego_duplo', '003_cracker_box', '054_softball', '056_tennis_ball', '057_racquetball', '077_rubiks_cube', '073-c_lego_duplo', '012_strawberry', '011_b

In [10]:
def gazebo_2_world(x,y):

    x_world= x+2.0
    y_world= y-1.0
    return (x_world,y_world)

def world_2_gazebo(x_world,y_world):

    x= ( x_world - 2.5)
    y= (-y_world + 1.5) 
    return (x , y)

In [11]:
z= 0 ## QUERY given Z where does the gaze ponits to wrt map


(trans,rot) = listener.lookupTransform('/map', '/head_rgbd_sensor_rgb_frame', rospy.Time(0))
e = np.rad2deg(tf.transformations.euler_from_quaternion(rot))
D_map=np.abs(trans[2]-z)/np.tan(np.deg2rad(e[0]-(-90)))
print ('x,y',D_map*-1* np.sin(np.deg2rad(e[2])), D_map* np.cos(np.deg2rad(e[2]))  )



('x,y', 1084.4964869409853, -13.028756816353374)


In [18]:
model_name= "073-g_lego_duplo"




x_gaz,y_gaz=world_2_gazebo(1,1)


spawn_object(model_name, model_name, x_gaz,  y_gaz, .21, 0)  ### ORigin in gazebo world is in (-2, 1 ) map coords
#print (x_gaz,y_gaz)

#spawn_object("073-g_lego_duplo", "073-g_lego_duplo", x_gaz, y_gaz, .51, 0)



[INFO] [1616453116.072740, 260.483000]: Spawn: 073-g_lego_duplo
[INFO] [1616453116.091966, 260.484000]: Waiting for service /gazebo/spawn_sdf_model
[INFO] [1616453116.102973, 260.485000]: Calling service /gazebo/spawn_sdf_model
[INFO] [1616453116.324936, 260.496000]: Spawn status: SpawnModel: Successfully spawned entity


In [17]:
delete_object(model_name)

In [21]:
head_pose = head.get_current_joint_values()
head_pose[0]=0.3
head_pose[1]=-0.4
head.set_joint_value_target(head_pose)
head.go()

True

In [15]:
def gaze_point_floor(x,y):
    
    arm_pose=arm.get_current_joint_values()
    arm_pose[0]=.1
    arm_pose[1]= -0.3
    arm.set_joint_value_target(arm_pose)
    arm.go()
    head_pose = head.get_current_joint_values()
    head_pose[0]=0.0
    head_pose[1]=0.0
    head.set_joint_value_target(head_pose)
    head.go()
    trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #
    np.rad2deg(head_pose)
    e =tf.transformations.euler_from_quaternion(rot)
    print('i am at',trans[:2],np.rad2deg(e)[2])
    print('gaze goal',x,y)
    #tf.transformations.euler_from_quaternion(rot)


    x_rob,y_rob,th_rob= trans[0], trans[1] ,  e[2]


    D_x=x_rob-x
    D_y=y_rob-y
    D_th= np.arctan2(D_y,D_x)
    print('relative to robot',(D_x,D_y,np.rad2deg(D_th)))

    pan_correct= (- th_rob + D_th + np.pi) % (2*np.pi)

    if(pan_correct > np.pi):
        pan_correct=-2*np.pi+pan_correct
    if(pan_correct < -np.pi):
        pan_correct=2*np.pi+pan_correct

    if ((pan_correct) > .5 * np.pi):
        print ('Exorcist alert')
        pan_correct=.5*np.pi
    head_pose[0]=pan_correct
    tilt_correct=np.arctan2(trans[2],np.linalg.norm((D_x,D_y)))

    head_pose [1]=-tilt_correct
    
    
    
    head.set_joint_value_target(head_pose)
    head.go()

In [16]:
gaze_point_floor(1,1)

('i am at', [-0.059838300843209236, 0.021300411920292965], -0.8390268622457722)
('gaze goal', 1, 1)
('relative to robot', (-1.0598383008432093, -0.978699588079707, -137.27930673822524))
